In [11]:
import pdftotext
import pandas as pd
import os

Идея в том, что нет необработанной грамматики, где количество страниц с обработанным текстом (будт то гугл предисловие / подписи / еще что) вдвое превышало бы количество страниц с картинками (те без текста)

In [12]:
def search_check(filename):
    
    with open(filename, "rb") as f:
        pdf = pdftotext.PDF(f)
        total_pages = len(pdf) # количество страниц
        empty_pages = 0 # количество пустых страниц 
        
        for page in pdf:
            if page == '\x0c' or page.startswith('Reproduced with permission of the copyright owner'):
                empty_pages += 1
        if empty_pages > round(total_pages/2):
            return'Not searchable'
        else:
            return'Searchable'

#### Найдем полный путь до файлов

In [13]:
directs = []  # полные пути к каждой pdf
for dir, subdir, files in os.walk("../Desktop/Language_Stuff/"):
    for file in files:
        if file.endswith('pdf'):
            directs.append(os.path.join(dir, file))

#### Проверка на искабельность каждой из pdf

In [14]:
ser = []  # есть поиск по тексту или нет 
for d in directs:
    ser.append(search_check(d))

#### создадим датафрейм

In [15]:
df = pd.DataFrame({
    'полный путь' : directs,
    'поиск по тексту' : ser
                  })
df.head(5)

,полный путь,поиск по тексту
0,../Desktop/Language_Stuff/African/Khoisan/Juǀ'...,Searchable
1,../Desktop/Language_Stuff/African/Khoisan/ǀXam...,Not searchable
2,../Desktop/Language_Stuff/African/Khoisan/!Xó...,Not searchable
3,../Desktop/Language_Stuff/African/Khoisan/Khoe...,Not searchable
4,../Desktop/Language_Stuff/African/Khoisan/Hadz...,Not searchable


In [16]:
import gspread
from gspread_dataframe import set_with_dataframe

# доступ к гугл-таблице
gc = gspread.service_account(filename='daring-agent-275412-2813966469f0.json')
sh = gc.open_by_key('1rKv6YwVua0GqiCpLyq1-wy71mcSir5ePGuBatYG6hWA')
worksheet = sh.get_worksheet(0) #-> 0 - первый лист, 1 - второй лист и т.д. 

your_dataframe = pd.DataFrame()
set_with_dataframe(worksheet, df) # экспорт

https://docs.google.com/spreadsheets/d/1rKv6YwVua0GqiCpLyq1-wy71mcSir5ePGuBatYG6hWA/edit#gid=0